In [1]:
import pandas as pd
import glob
import os

path = r'../data/raw/listings/'
all_files = glob.glob(os.path.join(path , "*.csv"))

dfs = list()

for file in all_files:
    filename = file[21:-4]
    # print(filename)
    df = pd.read_csv(file, index_col=None, dtype='object', header=0)
    df['city'] = filename.split(', ', 1)[0]
    df['state'] = filename.split(', ', 1)[1]
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

In [2]:
# Change States to proper one
df.loc[df['state'] == 'D.C., District of Columbia', 'state'] = 'Washington'
df.loc[df['state'] == 'OR, Oregon', 'state'] = 'Oregon'

In [3]:
df = df.drop(columns=['listing_url','scrape_id','last_scraped', 'description','neighborhood_overview','picture_url','host_url','host_name',
                                    'host_about','host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 
                                    'host_verifications','host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
                                    'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'minimum_minimum_nights', 'maximum_minimum_nights', 'amenities', 
                                    'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'bathrooms', 
                                    'review_scores_value', 'license', 'calendar_updated', 'has_availability', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review', 
                                    'last_review','instant_bookable', 'bathrooms_text', 'source', 'host_location', 'calendar_last_scraped', 'reviews_per_month', 
                                    'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 
                                    'calculated_host_listings_count_shared_rooms','calculated_host_listings_count',])
# 'review_scores_rating', 'amenities'                                  

In [4]:
df['price'] = df['price'].str.replace(',', '').str.replace('$', '')
df['price'] = df['price'].astype(float)
df['price']

C:\Users\wxl\AppData\Local\Temp\ipykernel_28100\835445966.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['price'] = df['price'].str.replace(',', '').str.replace('$', '')


0         100.0
1         100.0
2          66.0
3          45.0
4         100.0
          ...  
272283    172.0
272284    480.0
272285    173.0
272286     82.0
272287    108.0
Name: price, Length: 272288, dtype: float64

In [5]:
df.isna().sum()

id                                 0
name                              17
host_id                            0
host_since                        15
host_response_time             46135
host_response_rate             46135
host_acceptance_rate           37205
host_total_listings_count         15
latitude                           0
longitude                          0
property_type                      0
room_type                          0
accommodates                       0
bedrooms                       22266
beds                            3688
price                              0
minimum_nights                     0
maximum_nights                     0
availability_30                    0
availability_60                    0
availability_90                    0
availability_365                   0
number_of_reviews                  0
review_scores_rating           57490
review_scores_accuracy         58523
review_scores_cleanliness      58505
review_scores_checkin          58537
r

In [6]:
# change data type
df = df.astype({'latitude': 'float', 'longitude': 'float', 'accommodates': 'float', 'bedrooms': 'float',
                  'availability_30':'int', 'availability_60':'int', 'availability_90':'int', 'availability_365':'int', 
                  'minimum_nights': 'int', 'maximum_nights': 'int', 'number_of_reviews': 'int'})

In [7]:
numeric_cols=df.select_dtypes(include=['int', 'float']).columns
numeric_cols=numeric_cols.drop("longitude")
numeric_cols=numeric_cols.drop("latitude")

# keep the rows have non negative numbers
df = df.loc[(df[numeric_cols] >= 0).all(axis=1)]
df.head()

,id,name,host_id,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_total_listings_count,latitude,longitude,...,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,city,state
0,108061,Walk to stores/parks/downtown. Fenced yard/Pet...,320564,2010-12-16,within an hour,100%,71%,3,35.60670,-82.55563,...,246,92,4.51,4.56,4.7,4.85,4.8,4.84,Asheville,North Carolina
1,155305,Cottage! BonPaul + Sharky's Hostel,746673,2011-06-26,within an hour,100%,99%,11,35.57864,-82.59578,...,197,383,4.59,4.72,4.42,4.83,4.74,4.93,Asheville,North Carolina
2,156805,"Private Room ""Ader"" at BPS Hostel",746673,2011-06-26,within an hour,100%,99%,11,35.57864,-82.59578,...,0,67,4.52,4.73,4.43,4.76,4.61,4.84,Asheville,North Carolina
3,197263,Tranquil Room & Private Bath,961396,2011-08-12,within a few hours,100%,93%,2,35.57808,-82.63689,...,88,66,4.95,4.97,5.0,4.95,4.92,4.82,Asheville,North Carolina
4,209068,Terrace Cottage,1029919,2011-08-28,within a few hours,100%,44%,1,35.61856,-82.55276,...,231,60,4.87,4.84,4.84,5.0,4.98,4.93,Asheville,North Carolina


In [8]:
# save to csv zipped file
df.to_csv('../data/processed/clean_listings.gz', compression='gzip', index=False)

In [ ]:
# df=pd.read_csv("../data/processed/clean_listings.gz")
# df = df.astype({'id': 'object', 'host_id': 'object', 'latitude': 'float', 'longitude': 'float', 'accommodates': 'float', 'bedrooms': 'float', 'beds': 'float', 'price': 'float', 
#                'minimum_nights': 'int', 'maximum_nights': 'int', 'number_of_reviews': 'int', 'review_scores_rating': 'float',})
# df.head()
# df.dtypes
# print(len(df), len(df.columns))

In [ ]:
# listing_id	date	available	price	adjusted_price	minimum_nights	maximum_nights